In [1]:
from scipy.spatial import ConvexHull
from hdbscan import HDBSCAN
import shapefile as shp
import pandas as pd
import numpy as np

In [2]:
# Example usage
filename = 'layers/bayren.shp'

size = 250  # Ensure this is an integer

In [21]:
def loadfile(file):
    """
    load the shapefile and convert the coordinates to new dataframe
    :param file: path to the shapefile
    :return: xy-dataframe and the spatial projection
    """
    data = shp.Reader(file)
    proj = str(open(file.split('.')[0]+'.prj', 'r', encoding='utf-8').read())
    xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
    return pd.DataFrame(xy, columns=('x', 'y')), proj

def hdbscan_clustering(df, size):
    """
    perform the HDBSCAN clustering process
    :param df: input xy-dataframe
    :param size: min_cluster_size for the clustering algorithm
    :return: clustering label and clustering probabilities
    """
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df)
    return cluster.labels_ 

def cluster_new(filename, size, output=False):
    """
    Defines a boundary around cluster centers in a given point shapefile.
    :param filename: the given shapefile path
    :param size: min_cluster_size value for HDBSCAN clustering
    :param prob: probability threshold for boundary (default: 0)
    :param output: name for the output shapefile
    :return: original points df, output shape df
    """
    data, proj = loadfile(filename)
    data['label'] = hdbscan_clustering(data, size)
    
     

    data_filtered = data[data['label'] != -1]

    # 计算每个聚类的点数
    cluster_sizes = data_filtered['label'].value_counts()

    # 计算总点数
    total_points = cluster_sizes.sum()

    # 计算每个聚类的概率
    probabilities = cluster_sizes / total_points

    # 计算熵
    entropy = -np.sum(probabilities * np.log(probabilities))
    return entropy

In [25]:
for i in range(1000,1051,10):
    cluster = cluster_new(filename, i)
    print(cluster) 

3.547336510398879


KeyboardInterrupt: 

In [ ]:
import concurrent.futures
import numpy as np
import pandas as pd
import shapefile as shp
from hdbscan import HDBSCAN

def loadfile(file):
    data = shp.Reader(file)
    proj = str(open(file.split('.')[0]+'.prj', 'r', encoding='utf-8').read())
    xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
    return pd.DataFrame(xy, columns=('x', 'y')), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df[['x', 'y']])
    return cluster.labels_

def calculate_entropy(data):
    data_filtered = data[data['label'] != -1]
    cluster_sizes = data_filtered['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # Add eps for numerical stability

def cluster_new(filename, size):
    data, proj = loadfile(filename)
    data['label'] = hdbscan_clustering(data, size)
    return calculate_entropy(data)

def main():
    filename = "path_to_your_shapefile.shp"
    sizes = range(1000, 1011, 10)  # Sizes from 10 to 5000, stepping by 10
    entropy_results = {}

    # Using ProcessPoolExecutor to utilize multiple CPUs
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        # Submit all tasks to the executor
        future_to_size = {executor.submit(cluster_new, filename, size): size for size in sizes}
        
        # Collect results as they are completed
        for future in concurrent.futures.as_completed(future_to_size):
            size = future_to_size[future]
            try:
                entropy_results[size] = future.result()
            except Exception as exc:
                print(f"{size} generated an exception: {exc}")
    
    # Sorting results and printing
    for size in sorted(entropy_results.keys()):
        print(f"Size: {size}, Entropy: {entropy_results[size]}")

if __name__ == "__main__":
    main()

In [31]:
import pandas as pd
import numpy as np
import shapefile as shp
from hdbscan import HDBSCAN
import logging

def loadfile(file):
    # 尝试使用with语句确保文件正确关闭
    with shp.Reader(file) as data:
        proj_file = file.split('.')[0]+'.prj'
        with open(proj_file, 'r', encoding='utf-8') as f:
            proj = str(f.read())
        xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
        return pd.DataFrame(xy, columns=('x', 'y')), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df)
    return cluster.labels_

def calculate_entropy(data):
    data_filtered = data[data['label'] != -1]
    cluster_sizes = data_filtered['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # Add eps for numerical stability

def cluster_new(filename, size):
    data, proj = loadfile(filename)
    data['label'] = hdbscan_clustering(data, size)
    return calculate_entropy(data)

def run_sequential(filename, start, end, step):
    results = {}
    for size in range(start, end, step):
        try:
            entropy = cluster_new(filename, size)
            results[size] = entropy
            print(f"Size {size}: Entropy {entropy}")
        except Exception as e:
            print(f"Error processing size {size}: {str(e)}")
    return results

if __name__ == "__main__":
    filename = 'layers/bayren.shp'
    start, end, step = 100, 501, 10
    run_sequential(filename, start, end, step)

Size 100: Entropy 6.5769836549003
Size 110: Entropy 6.492330841394348
Size 120: Entropy 6.397774715709114
Size 130: Entropy 6.331139974425614
Size 140: Entropy 6.229904211785767
Size 150: Entropy 6.149759620022432
Size 160: Entropy 6.109564133836278
Size 170: Entropy 6.016144340615692
Size 180: Entropy 6.119316799226815
Size 190: Entropy 6.058884078343684
Size 200: Entropy 5.705961959742417
Size 210: Entropy 5.688512788215408
Size 220: Entropy 5.586786771545737


KeyboardInterrupt: 

In [39]:
import pandas as pd
import numpy as np
import shapefile as shp
from hdbscan import HDBSCAN
import logging

def loadfile(file):
    # 使用 with 语句确保文件正确关闭
    with shp.Reader(file) as data:
        proj_file = file.split('.')[0]+'.prj'
        with open(proj_file, 'r', encoding='utf-8') as f:
            proj = str(f.read())
        xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
        return pd.DataFrame(xy, columns=['x', 'y']), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df[['x', 'y']])
    return cluster.labels_

def calculate_entropy(data):
    if data.empty:
        return 0  # 返回0熵值，如果没有数据点符合条件
    cluster_sizes = data['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # 为数值稳定性添加 eps

def cluster_new(filename, size):
    data, proj = loadfile(filename)
    data['label'] = hdbscan_clustering(data, size)
    
    # 筛选出聚类大小大于等于200的聚类,这个数值如何求得可能是我们未来的一个关键讨论,或者说是直接通过经验?
    cluster_counts = data['label'].value_counts()
    clusters_to_keep = cluster_counts[cluster_counts >= 500].index
    data = data[data['label'].isin(clusters_to_keep)]

    return calculate_entropy(data)

def run_sequential(filename, start, end, step):
    results = {}
    for size in range(start, end, step):
        try:
            entropy = cluster_new(filename, size)
            results[size] = entropy
            print(f"Size {size}: Entropy {entropy}")
        except Exception as e:
            print(f"Error processing size {size}: {str(e)}")
    return results

if __name__ == "__main__":
    filename = 'layers/bayren.shp'
    start, end, step = 10, 501, 10
    run_sequential(filename, start, end, step)

Size 10: Entropy 1.8294758850038906
Size 20: Entropy 1.9973332557479302
Size 30: Entropy 2.251540013851125
Size 40: Entropy 2.3376793555768
Size 50: Entropy 2.2998386593962596
Size 60: Entropy 2.5183966622867673
Size 70: Entropy 2.473883176945511
Size 80: Entropy 2.481547351014081
Size 90: Entropy 2.6196993966268285
Size 100: Entropy 2.6613488624212245
Size 110: Entropy 2.603152613107766
Size 120: Entropy 2.7140087458870363
Size 130: Entropy 2.7637697119264217
Size 140: Entropy 2.79488661017308
Size 150: Entropy 2.890433617284593
Size 160: Entropy 2.9255407879539934
Size 170: Entropy 2.8939376052026193
Size 180: Entropy 2.9245076507460452
Size 190: Entropy 2.9204494053997094
Size 200: Entropy 2.9298151587625263
Size 210: Entropy 2.9138290737004984
Size 220: Entropy 2.9646390637280957
Size 230: Entropy 2.9163053438945745
Size 240: Entropy 2.974992317891591
Size 250: Entropy 3.065277205227973
Size 260: Entropy 3.0751058035741328
Size 270: Entropy 3.028958579073443
Size 280: Entropy 2.990

In [37]:
import pandas as pd
import numpy as np
import shapefile as shp
from hdbscan import HDBSCAN
import logging

def loadfile(file):
    # 使用 with 语句确保文件正确关闭
    with shp.Reader(file) as data:
        proj_file = file.split('.')[0]+'.prj'
        with open(proj_file, 'r', encoding='utf-8') as f:
            proj = str(f.read())
        xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
        return pd.DataFrame(xy, columns=['x', 'y']), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df[['x', 'y']])
    return cluster.labels_

def calculate_entropy(data):
    if data.empty:
        return 0  # 返回0熵值，如果没有数据点符合条件
    cluster_sizes = data['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # 为数值稳定性添加 eps

def cluster_new(filename, size):
    data, proj = loadfile(filename)
    data['label'] = hdbscan_clustering(data, size)
    
    # 筛选出聚类大小大于等于200的聚类,这个数值如何求得可能是我们未来的一个关键讨论,或者说是直接通过经验?
    cluster_counts = data['label'].value_counts()
    clusters_to_keep = cluster_counts[cluster_counts >= 1000].index
    data = data[data['label'].isin(clusters_to_keep)]

    return calculate_entropy(data)

def run_sequential(filename, start, end, step):
    results = {}
    for size in range(start, end, step):
        try:
            entropy = cluster_new(filename, size)
            results[size] = entropy
            print(f"Size {size}: Entropy {entropy}")
        except Exception as e:
            print(f"Error processing size {size}: {str(e)}")
    return results

if __name__ == "__main__":
    filename = 'layers/bayren.shp'
    start, end, step = 10, 501, 50
    run_sequential(filename, start, end, step)

Size 10: Entropy 0.6366429495474949
Size 60: Entropy 1.301865870028255
Size 110: Entropy 1.4419651968568798
Size 160: Entropy 1.7617392582917921
Size 210: Entropy 1.8612254978224767
Size 260: Entropy 2.2313250941115097
Size 310: Entropy 2.195873613381045
Size 360: Entropy 2.2878432768549217
Size 410: Entropy 2.4526528694382614
Size 460: Entropy 2.4722568308340156


In [35]:
import pandas as pd
import numpy as np
import shapefile as shp
from hdbscan import HDBSCAN
import logging

def loadfile(file):
    # Ensuring that files are properly closed after opening
    with shp.Reader(file) as data:
        proj_file = file.split('.')[0] + '.prj'
        with open(proj_file, 'r', encoding='utf-8') as f:
            proj = str(f.read())
        xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
        return pd.DataFrame(xy, columns=['x', 'y']), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df[['x', 'y']])
    return cluster.labels_, cluster.probabilities_

def calculate_entropy(data):
    if data.empty:
        return 0  # Returning entropy of 0 if there are no data points after filtering
    data_filtered = data[data['label'] != -1]
    cluster_sizes = data_filtered['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # Adding eps for numerical stability

def cluster_new(filename, size, min_prob):
    data, proj = loadfile(filename)
    labels, probabilities = hdbscan_clustering(data, size)
    data['label'] = labels
    data['probabilities'] = probabilities  # Storing probabilities in the DataFrame

    # Filtering data based on probability threshold
    data_filtered = data[(data['probabilities'] >= min_prob) & (data['label'] != -1)]
    return calculate_entropy(data_filtered)

def run_sequential(filename, start, end, step, min_prob):
    results = {}
    for size in range(start, end, step):
        try:
            entropy = cluster_new(filename, size, min_prob)
            results[size] = entropy
            print(f"Size {size}: Entropy {entropy}")
        except Exception as e:
            print(f"Error processing size {size}: {str(e)}")
    return results

if __name__ == "__main__":
    filename = 'layers/bayren.shp'
    start, end, step = 10, 101, 10
    min_prob = 0.7  # Probability threshold
    run_sequential(filename, start, end, step, min_prob)

Size 100: Entropy 6.521866757013709
Size 130: Entropy 6.30626843660319
Size 160: Entropy 6.093844347347492
Size 190: Entropy 6.111320340928155
Size 220: Entropy 5.538104363058853
Size 250: Entropy 5.22379706332438
Size 280: Entropy 5.182970547394489
Size 310: Entropy 5.14286040027156


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import shapefile as shp
from hdbscan import HDBSCAN
import logging

def loadfile(file):
    # Ensuring that files are properly closed after opening
    with shp.Reader(file) as data:
        proj_file = file.split('.')[0] + '.prj'
        with open(proj_file, 'r', encoding='utf-8') as f:
            proj = str(f.read())
        xy = [geo['geometry']['coordinates'] for geo in data.__geo_interface__['features']]
        return pd.DataFrame(xy, columns=['x', 'y']), proj

def hdbscan_clustering(df, size):
    if not isinstance(size, int):
        raise ValueError("Min cluster size must be an integer!")
    cluster = HDBSCAN(min_cluster_size=size).fit(df[['x', 'y']])
    return cluster.labels_, cluster.probabilities_

def calculate_entropy(data):
    if data.empty:
        return 0  # Returning entropy of 0 if there are no data points after filtering
    data_filtered = data[data['label'] != -1]
    cluster_sizes = data_filtered['label'].value_counts()
    total_points = cluster_sizes.sum()
    probabilities = cluster_sizes / total_points
    return -np.sum(probabilities * np.log(probabilities + np.finfo(float).eps))  # Adding eps for numerical stability

def cluster_new(filename, size, min_prob):
    data, proj = loadfile(filename)
    labels, probabilities = hdbscan_clustering(data, size)
    data['label'] = labels
    data['probabilities'] = probabilities  # Storing probabilities in the DataFrame

    # Filtering data based on probability threshold
    data_filtered = data[(data['probabilities'] >= min_prob) & (data['label'] != -1)]
    return calculate_entropy(data_filtered)

def run_sequential(filename, start, end, step, min_prob):
    results = {}
    for size in range(start, end, step):
        try:
            entropy = cluster_new(filename, size, min_prob)
            results[size] = entropy
            print(f"Size {size}: Entropy {entropy}")
        except Exception as e:
            print(f"Error processing size {size}: {str(e)}")
    return results

if __name__ == "__main__":
    filename = 'layers/bayren.shp'
    start, end, step = 10, 101, 10
    min_prob = 0.7  # Probability threshold
    run_sequential(filename, start, end, step, min_prob)

Size 10: Entropy 8.499139793585032
Size 20: Entropy 8.055797669943384
Size 30: Entropy 7.772642063155997
Size 40: Entropy 7.548482444329723
Size 50: Entropy 7.401359266260523
Size 60: Entropy 6.983034014565659
Size 70: Entropy 7.077685972625366
